# Generating Qubit Hamiltonians

In [10]:
from utility import *
import numpy as np 

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [11]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(h2))

H:
-0.3276081896748093 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099508 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109106 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109106 [Z3]


This Qubit Hamiltonian encodes all of the $2^N$ eigenstates and eigenenergies, where $N=4$ is the number of qubits. 
In principle, one can represent the Hamiltonian as a matrix with $2^N$ dimensions and find the eigenvalues classically through diagonalization, but the cost of this approach clearly grows exponentially with $N$. 

Still, we can check that this approach indeed recovers the ground state energy against method in S1. 

In [12]:
# Defining pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

# Build matrix representiation of the Hamiltonian H
n_qubits = openfermion.count_qubits(h2)
h2_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in h2.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h2_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h2_matrix)
print("The ground state energy from S1: ")
obtain_PES('h2', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
converged SCF energy = -1.06610864931794
E = -1.101150330232619 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-1.10115033 -0.74587179 -0.74587179 -0.74587179 -0.60860671 -0.60860671
 -0.58166697 -0.58166697 -0.35229063 -0.06021529 -0.06021529 -0.05994379
 -0.05994379  0.03904763  0.50196598  0.52917721]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [13]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337645 [] +
0.1967905834854702 [X0] +
-0.5350572998841723 [Z0]


We can verify that this new Hamiltonian still encodes the ground state energy. 

In [14]:
# Building the matrix representation of the effective Hamiltonian
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("The eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]


# Same exercise for LiH

In [15]:
qubit_transf = 'jw' # Jordan-Wigner transformations
lih_ham = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(lih_ham))

H:
-3.934441956757906 [] +
-0.00792332115785017 [X0 X1 Y2 Y3] +
-0.003414532358015887 [X0 X1 Y2 Z3 Z4 Y5] +
-0.002746861320316999 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.003414532358015887 [X0 X1 X3 X4] +
-0.0027468613203169984 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.004864778381761382 [X0 X1 Y4 Y5] +
-0.002296316587299585 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.002296316587299585 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.0024727061683852865 [X0 X1 Y6 Y7] +
-0.002472706168385286 [X0 X1 Y8 Y9] +
-0.0017744350099502802 [X0 X1 Y10 Y11] +
0.00792332115785017 [X0 Y1 Y2 X3] +
0.003414532358015887 [X0 Y1 Y2 Z3 Z4 X5] +
0.002746861320316999 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.003414532358015887 [X0 Y1 Y3 X4] +
-0.0027468613203169984 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.004864778381761382 [X0 Y1 Y4 X5] +
0.002296316587299585 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.002296316587299585 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.0024727061683852865 [X0 Y1 Y6 X7] +
0.002472706168385286 [X0 Y1 Y8 X9] +
0.0017744

In [16]:
# Defining pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

# Build matrix representiation of the Hamiltonian LiH
n_qubits = openfermion.count_qubits(lih_ham)
print("Number of qubits =", n_qubits)
h2_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in lih_ham.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h2_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h2_matrix)
print("The ground state energy from S1: ")
obtain_PES('lih', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

Number of qubits = 12
The ground state energy from S1: 
converged SCF energy = -7.76736213574856
E = -7.784021320447714 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-7.78446028 -7.71129396 -7.71129396 ...  1.58753163  1.86602235
  2.08130331]


In [19]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(lih_ham, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf)))

The effective Hamiltonian:
 -3.7779692011782013 [] +
0.040696065058758175 [X0] +
-0.002366478148278927 [X0 X1 Y2 Y3] +
0.007237092209029691 [X0 X1 Y2 Z3 Z6 Y7] +
0.0016354047160372 [X0 X1 Z2 X3 Z4 Z5 Z7] +
-0.0031369437172315525 [X0 X1 Z2 Z4 Z5 Z6 X7] +
0.007237092209029691 [X0 X1 X3 X6] +
-0.0016354047160372 [X0 X1 X3 Z6 Z7] +
0.006795526682425028 [X0 X1 X4] +
0.006795526682425023 [X0 X1 X5] +
-0.03060391141885304 [X0 X1 Y6 Y7] +
0.0031369437172315525 [X0 X1 X7] +
0.0016354047160372 [X0 Y1 Y2] +
0.002366478148278927 [X0 Y1 Y2 X3] +
-0.0016354047160372 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-0.007237092209029691 [X0 Y1 Y2 Z3 Z6 X7] +
0.0016354047160372 [X0 Y1 Z2 Y3 Z4 Z5 Z7] +
-0.003136943717231552 [X0 Y1 Z2 Z3 Y6] +
0.003136943717231552 [X0 Y1 Z2 Z4 Z5 Y6 Z7] +
-0.0031369437172315525 [X0 Y1 Z2 Z4 Z5 Z6 Y7] +
0.007237092209029691 [X0 Y1 Y3 X6] +
-0.0016354047160372 [X0 Y1 Y3 Z6 Z7] +
0.006795526682425028 [X0 Y1 Y4] +
0.006795526682425023 [X0 Y1 Y5] +
0.03060391141885304 [X0 Y1 Y6 X7] +
0.00313694371